In [185]:
import pandas as pd
import numpy as np
import os
import neurokit2 as nk
import seaborn as sns
import matplotlib.pyplot as plt
import wfdb
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
import random
import json
import os
from sklearn.metrics import accuracy_score
from mealpy.evolutionary_based.GA import BaseGA
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from mealpy import FloatVar
from mealpy import IntegerVar
import time
import re
import torch.nn.functional as F

In [186]:
file_path = Path('C:/Users/vinay/Downloads/mit-bih-arrhythmia-database-1.0.0/mit-bih-arrhythmia-database-1.0.0')

In [187]:
data_files=[]
annot_files=[]
for file in os.listdir(file_path):
    if('.dat' in file):
        data_files.append(file[:-4])
    elif('.atr' in file):
        annot_files.append(file[:-4])

In [188]:
char_to_int = {}
count = 0 

for file in annot_files:
    path_file = os.path.join(file_path, file)
    annotation = wfdb.rdann(path_file, 'atr') 
    
    for symbol in annotation.symbol:
        if symbol not in char_to_int: 
            char_to_int[symbol] = count
            count += 1 

print(char_to_int)

{'+': 0, 'N': 1, 'A': 2, 'V': 3, '~': 4, '|': 5, 'Q': 6, '/': 7, 'f': 8, 'x': 9, 'F': 10, 'j': 11, 'L': 12, 'a': 13, 'J': 14, 'R': 15, '[': 16, '!': 17, ']': 18, 'E': 19, 'S': 20, '"': 21, 'e': 22}


In [189]:
for file in annot_files:
    path_file = os.path.join(file_path, file)
    annotation = wfdb.rdann(path_file, 'atr')
    unique, counts = np.unique(annotation.symbol, return_counts=True)
    print(file, dict(zip(unique, counts)), counts.sum(), len(annotation.sample))

100 {'+': 1, 'A': 33, 'N': 2239, 'V': 1} 2274 2274
101 {'+': 1, 'A': 3, 'N': 1860, 'Q': 2, '|': 4, '~': 4} 1874 1874
102 {'+': 5, '/': 2028, 'N': 99, 'V': 4, 'f': 56} 2192 2192
103 {'+': 1, 'A': 2, 'N': 2082, '~': 6} 2091 2091
104 {'+': 45, '/': 1380, 'N': 163, 'Q': 18, 'V': 2, 'f': 666, '~': 37} 2311 2311
105 {'+': 1, 'N': 2526, 'Q': 5, 'V': 41, '|': 30, '~': 88} 2691 2691
106 {'+': 41, 'N': 1507, 'V': 520, '~': 30} 2098 2098
107 {'+': 1, '/': 2078, 'V': 59, '~': 2} 2140 2140
108 {'+': 1, 'A': 4, 'F': 2, 'N': 1739, 'V': 17, 'j': 1, 'x': 11, '|': 8, '~': 41} 1824 1824
109 {'+': 1, 'F': 2, 'L': 2492, 'V': 38, '~': 2} 2535 2535
111 {'+': 1, 'L': 2123, 'V': 1, '~': 8} 2133 2133
112 {'+': 1, 'A': 2, 'N': 2537, '~': 10} 2550 2550
113 {'+': 1, 'N': 1789, 'a': 6} 1796 1796
114 {'+': 3, 'A': 10, 'F': 4, 'J': 2, 'N': 1820, 'V': 43, '|': 1, '~': 7} 1890 1890
115 {'+': 1, 'N': 1953, '|': 6, '~': 2} 1962 1962
116 {'+': 1, 'A': 1, 'N': 2302, 'V': 109, '~': 8} 2421 2421
117 {'+': 1, 'A': 1, 'N': 153

In [190]:
all_signals = []
all_labels = []

for i in range(48):
    data, field = wfdb.rdsamp(os.path.join(file_path, data_files[i]))
    data = data[:, 0]
    
    annot = wfdb.rdann(os.path.join(file_path, annot_files[i]), 'atr')
    segmented_signals = [data[max(0, peak - 64):min(len(data), peak + 64)] for peak in annot.sample]
    
    segmented_array = np.array([
        np.pad(signal, (0, 128 - len(signal)), mode='edge') if len(signal) < 128 else signal
        for signal in segmented_signals
    ])
    
    labels = annot.symbol[:len(segmented_array)]  

    all_signals.append(segmented_array)
    all_labels.append(labels)

In [191]:
f_X_train, f_X_test, f_y_train, f_y_test = [], [], [], []
for i in range(48):
    X = np.array(all_signals[i]).reshape(-1, 128)
    y = np.array(all_labels[i]).flatten()
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.25,
        random_state=42
    )
    f_X_train.append(X_train)
    f_X_test.append(X_test)
    f_y_train.extend(y_train)  
    f_y_test.extend(y_test)

In [192]:
f_X_train = np.vstack(f_X_train).reshape(-1, 128, 1)
f_X_test = np.vstack(f_X_test).reshape(-1, 128, 1)
y_train_int = [char_to_int.get(char, -1) for char in f_y_train]
y_test_int = [char_to_int.get(char, -1) for char in f_y_test]
y_train_int = [y for y in y_train_int if y != -1]
y_test_int = [y for y in y_test_int if y != -1]
num_classes = len(char_to_int)
f_y_train = to_categorical(y_train_int, num_classes=num_classes)
f_y_test = to_categorical(y_test_int, num_classes=num_classes)
print(f"Final Train Shape: {f_X_train.shape}, Test Shape: {f_X_test.shape}")
print(f"Final Train Labels Shape: {f_y_train.shape}, Test Labels Shape: {f_y_test.shape}")

Final Train Shape: (84470, 128, 1), Test Shape: (28177, 128, 1)
Final Train Labels Shape: (84470, 23), Test Labels Shape: (28177, 23)


In [193]:
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

In [208]:
class ECGModel(nn.Module):
    def __init__(self, config, num_classes):
        super(ECGModel, self).__init__()

        feature_extractor = config[8]
        sequence_model = config[9]
        num_cnn_layers = int(config[0])
        num_rnn_layers = int(config[1])
        dropout = config[3]
        num_filters = int(config[4])
        kernel_size = int(config[5])
        stride = int(config[6])
        hidden_size = int(config[10])
        fc_neurons = int(config[11])

        self.input_size = 128  # Set initial input size
        layers = []
        in_channels = 1

        for _ in range(num_cnn_layers):
            print(f"🔹 Before Conv Layer: {self.input_size}")
    
            layers.append(nn.Conv1d(in_channels, num_filters, kernel_size=kernel_size, stride=stride, padding=1))
            layers.append(nn.BatchNorm1d(num_filters))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_channels = num_filters

            output_size = ((self.input_size - kernel_size + 2 * 1) // stride) + 1
            print(f"🔹 After Conv Layer: {output_size}")

            self.input_size = output_size

        self.feature_extractor = nn.Sequential(*layers)

        # Final feature output size
        self.feature_output_size = self.input_size  

        # RNN Configuration
        if feature_extractor == 1:
            self.rnn = nn.LSTM(input_size=self.feature_output_size, hidden_size=hidden_size, num_layers=num_rnn_layers, batch_first=True, dropout=dropout)
        else:
            self.rnn = None

        # Choose between LSTM or GRU
        if sequence_model == 0:
            self.rnn_layer = nn.LSTM(input_size=self.feature_output_size, hidden_size=hidden_size, num_layers=num_rnn_layers, batch_first=True, dropout=dropout) 
        else:
            self.rnn_layer = nn.GRU(input_size=self.feature_output_size, hidden_size=hidden_size, num_layers=num_rnn_layers, batch_first=True, dropout=dropout)

        # Fully Connected Layers
        self.fc = nn.Linear(hidden_size, fc_neurons)
        self.output = nn.Linear(fc_neurons, num_classes)

        self.apply(self.init_weights)

    def forward(self, x):
        print(f"🔍 Before CNN: {x.shape}")  # Expect (batch, 1, seq_len)

        x = self.feature_extractor(x)
        print(f"🔍 After CNN: {x.shape}")  # Expect (batch, num_filters, seq_len)

        # Ensure correct feature order for LSTM
        x = x.permute(0, 2, 1) if self.feature_extractor else x  # Now should be (batch, seq_len, features)
        print(f"🔍 After Permute (before RNN): {x.shape}")

        assert x.shape[-1] == self.feature_output_size, f"🚨 Mismatch: Expected {self.feature_output_size}, but got {x.shape[-1]}"

        x, _ = self.rnn_layer(x)  # Pass through LSTM
        print(f"🔍 After RNN: {x.shape}")  # Expect (batch, seq_len, hidden_size)

        x = self.fc(x[:, -1, :])  # Take last time step
        x = self.output(x)
        return x


    def init_weights(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv1d):
            nn.init.xavier_uniform_(m.weight)
        elif isinstance(m, nn.LSTM) or isinstance(m, nn.GRU):
            for name, param in m.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.constant_(param, 0)


In [195]:
def format_filename(config):
    formatted_config = "_".join([f"{x:.4f}" if isinstance(x, float) else str(x) for x in config])
    formatted_config = re.sub(r'[^\w\-_]', '', formatted_config)
    return formatted_config

In [196]:
def convert_to_serializable(obj):
    """ Converts NumPy arrays and other non-serializable objects to lists for JSON serialization """
    if isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert NumPy array to a list
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]  # Recursively process lists
    return obj  # Return normal values unchanged

In [197]:
def measure_latency(model, dataloader, device="cpu"):
    model.eval()
    total_time = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            batch_size = inputs.shape[0]
            start_time = time.perf_counter()
            _ = model(inputs)
            end_time = time.perf_counter()
            total_time += (end_time - start_time) * 1000
            num_samples += batch_size

    return total_time / num_samples

In [198]:
def check_input_shape(tensor, expected_features):
    print(f"🔥 Input tensor shape: {tensor.shape}")  # Debugging
    if tensor.shape[-1] != expected_features:
        raise ValueError(f"Expected input features: {expected_features}, but got: {tensor.shape[-1]}")
    return tensor

In [205]:
def train_and_evaluate(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Create a temporary model to get feature size
    num_classes = f_y_train.shape[1]
    temp_model = ECGModel(config, num_classes)

    print(f"✅ CNN Output Size: {temp_model.feature_output_size}")
    expected_features = temp_model.feature_output_size  # ✅ Fixed this line

    # Check dataset shape before passing
    print(f"✅ Raw X_train shape: {f_X_train.shape}")
    
    X_train, X_test = (
        check_input_shape(torch.tensor(f_X_train, dtype=torch.float32).permute(0, 2, 1), expected_features),
        check_input_shape(torch.tensor(f_X_test, dtype=torch.float32).permute(0, 2, 1), expected_features),
    )

    print(f"✅ Adjusted X_train shape: {X_train.shape}")  # Should match `expected_features`
    y_train, y_test = torch.tensor(f_y_train, dtype=torch.long), torch.tensor(f_y_test, dtype=torch.long)

    batch_size = int(config[7]) if len(config) > 7 else 64
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

    # Now create the actual model
    model = ECGModel(config, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config[2])

    num_epochs = 10
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(train_loader):.4f}")

    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            true_labels = labels.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(true_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    latency = measure_latency(model, test_loader, device)
    fitness = accuracy - (latency / 100)

    model_name = f"model_{format_filename(config)}.pt"
    torch.save(model.state_dict(), f"models/{model_name}")

    serializable_config = convert_to_serializable(config)
    result = {"config": serializable_config, "accuracy": accuracy, "latency": latency, "fitness": fitness}
    with open(f"results/{model_name}.json", "w") as f:
        json.dump(result, f, indent=4)

    print(f"✅ Config: {serializable_config} -> Accuracy: {accuracy:.4f}, Latency: {latency:.2f}ms, Fitness: {fitness:.4f} | Model Saved: {model_name}")
    return fitness


In [209]:
bounds = [
    IntegerVar(lb=1, ub=5),
    IntegerVar(lb=1, ub=5),
    FloatVar(lb=0.0001, ub=0.01),
    FloatVar(lb=0.1, ub=0.5),
    IntegerVar(lb=32, ub=128),
    IntegerVar(lb=3, ub=7),
    IntegerVar(lb=1, ub=2),
    IntegerVar(lb=16, ub=128),
    IntegerVar(lb=0, ub=1),
    IntegerVar(lb=0, ub=1),
    IntegerVar(lb=64, ub=512),
    IntegerVar(lb=32, ub=256),
]

problem_dict = {"bounds": bounds, "obj_func": train_and_evaluate, "minmax": "max"}

ga_model = BaseGA(epoch=10, pop_size=5, pc=0.9, pm=0.05)
best_solution = ga_model.solve(problem_dict)

best_architecture = best_solution.solution
best_fitness = best_solution.target.fitness

best_model_info = {"best_architecture": best_architecture, "best_fitness": best_fitness}
with open("results/best_model.json", "w") as f:
    json.dump(best_model_info, f, indent=4)

print(f"\n🏆 Best Model Configuration: {best_architecture}, Fitness Score: {best_fitness}")


🔹 Before Conv Layer: 128
🔹 After Conv Layer: 64
🔹 Before Conv Layer: 64
🔹 After Conv Layer: 32
✅ CNN Output Size: 32
✅ Raw X_train shape: (84470, 128, 1)
🔥 Input tensor shape: torch.Size([84470, 1, 128])


ValueError: Expected input features: 32, but got: 128